# Perbandingan Model Peringkasan Ekstraktif

Notebook ini mengimplementasikan studi komparatif berbagai model pembelajaran mesin untuk peringkasan ekstraktif. Kami memperlakukan tugas peringkasan sebagai masalah klasifikasi biner pada tingkat kalimat.

**Tujuan:** Memprediksi apakah sebuah kalimat termasuk dalam ringkasan (`Label_Final = 1`) atau tidak (`0`) berdasarkan fitur yang diekstraksi.

**Model yang Dibandingkan:**
1. Support Vector Machine (SVM) - *Minat Utama*
2. Random Forest (RF)
3. Logistic Regression (LR)
4. K-Nearest Neighbors (KNN)
5. Naive Bayes (NB)

**Metodologi:**
- **Baseline:** Hiperparameter default.
- **Optimasi:** Penyetelan hiperparameter menggunakan Optuna.
- **Evaluasi:** Validasi Silang K-Fold Terstratifikasi (k=5).
- **Metrik:** Akurasi, Presisi, Recall, F1-Score, Cohen's Kappa.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from src.data_preprocessing import prepare_data
from src.model_trainer import get_models, evaluate_model
from src.optimizer import optimize_model

# Set plot style
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

## 1. Pemuatan dan Pra-pemrosesan Data

Kami memuat dataset, menangani nilai yang hilang, dan melakukan penskalaan fitur. Penskalaan sangat penting untuk model berbasis jarak seperti SVM dan KNN.

In [ ]:
# Load and prepare data
X, y = prepare_data('korpus_mentah_new.csv')

print("Feature shape:", X.shape)
print("Target distribution:\n", y.value_counts(normalize=True))
X.head()

## 2. Evaluasi Model Baseline

Kami mengevaluasi kelima model menggunakan hiperparameter default mereka untuk menetapkan kinerja dasar (baseline).

In [ ]:
baseline_models = get_models()
baseline_results = []

print("Running Baseline Evaluation...")
for name, model in baseline_models.items():
    print(f"Evaluating {name}...")
    metrics = evaluate_model(model, X, y)
    metrics['Model'] = name
    metrics['Type'] = 'Baseline'
    baseline_results.append(metrics)

df_baseline = pd.DataFrame(baseline_results)
df_baseline

## 3. Optimasi Hiperparameter (Optuna)

Kami menggunakan Optuna untuk menemukan hiperparameter optimal bagi setiap model. Kami memaksimalkan **F1-Score**, karena metrik ini menyeimbangkan Presisi dan Recall, yang penting untuk tugas peringkasan (kami ingin memilih kalimat yang relevan tanpa melewatkan terlalu banyak).

In [ ]:
optimized_results = []
best_params_log = {}

print("Running Optuna Optimization (this may take a while)...")
for name in baseline_models.keys():
    print(f"Optimizing {name}...")
    # Run optimization (50 trials per model)
    best_model, best_params, best_score = optimize_model(name, X, y, n_trials=50)
    
    best_params_log[name] = best_params
    print(f"  Best F1: {best_score:.4f}")
    
    # Evaluate the optimized model using the same CV strategy
    metrics = evaluate_model(best_model, X, y)
    metrics['Model'] = name
    metrics['Type'] = 'Optimized'
    optimized_results.append(metrics)

df_optimized = pd.DataFrame(optimized_results)
df_optimized

### Hiperparameter Terbaik yang Ditemukan

In [ ]:
for name, params in best_params_log.items():
    print(f"--- {name} ---")
    print(params)
    print("")

## 4. Perbandingan & Analisis

Kami menggabungkan hasil untuk membandingkan peningkatan kinerja.

In [ ]:
df_final = pd.concat([df_baseline, df_optimized], ignore_index=True)
df_final.sort_values(by=['Model', 'Type'], inplace=True)
df_final

### Visualisasi: Perbandingan F1-Score

In [ ]:
plt.figure(figsize=(14, 7))
sns.barplot(data=df_final, x='Model', y='F1-Score', hue='Type', palette='viridis')
plt.title('Model Comparison: Baseline vs Optimized (F1-Score)', fontsize=16)
plt.ylim(0, 1.0)
plt.legend(title='Configuration', loc='lower right')
plt.grid(axis='y', alpha=0.3)
plt.show()

### Visualisasi: Perbandingan Cohen's Kappa
Cohen's Kappa adalah metrik yang kuat untuk kesepakatan antar-penilai, berguna di sini untuk melihat seberapa jauh lebih baik model dibandingkan dengan peluang acak.

In [ ]:
plt.figure(figsize=(14, 7))
sns.barplot(data=df_final, x='Model', y='Kappa', hue='Type', palette='magma')
plt.title("Model Comparison: Baseline vs Optimized (Cohen's Kappa)", fontsize=16)
plt.ylim(0, 1.0)
plt.legend(title='Configuration', loc='lower right')
plt.grid(axis='y', alpha=0.3)
plt.show()

## 5. Kesimpulan

Tabel dan grafik di atas merangkum kinerja SVM dibandingkan dengan Random Forest, Logistic Regression, KNN, dan Naive Bayes.

**Observasi Utama:**
1. **Dampak Optimasi:** Periksa apakah Optuna meningkatkan skor secara signifikan. SVM dan KNN sering kali paling diuntungkan dari penskalaan dan penyetelan.
2. **Model Terbaik:** Identifikasi model mana yang mencapai F1-Score dan Kappa tertinggi.
3. **Konsistensi:** Perhatikan stabilitas hasil.

Analisis ini membantu memilih model yang paling tangguh untuk alur kerja peringkasan teks.